## How To Perform an Resonator Spectroscopy Experiment

### Prerequisites
This guide assumes you have a configured `DeviceSetup` as well as `Qubit` objects with assigned parameters. Please see these guides (add links) if you need to create your setup and qubits for the first time. However, you can also run this notebook "as is" using an emulated session. If you are just getting started with the LabOne Q Applications Library, please don't hesitate to reach out to us at info@zhinst.com.

### Background
In this how-to guide, you'll perform a measurement to find the optimal response frequency of a resonator to maximize signal-to-noise for a qubit measurement. The LabOne Q Applications Library provides different experiment workflows to achieve this task. You can either just sweep the frequency of a measure-pulse using the `resonator_spectroscopy` experiment workflow, or perform a 2D sweep to optimize at the same time the amplitude of the measure-pulse (the `resonator_spectroscopy_amplitude` experiment workflow). The measure-pulse modulated at different frequencies will be sent on a readout line traversing multiple resonators each coupled with one qubit.

When isolated, a superconducting resonator can be fully described simply by the magnetic flux threaded through the inductor $\phi$ and the difference $n$ in the number of charges on the two plates of the capacitor in units of Copper pairs. In this case, the resulting Hamiltonian will be

$$H = \frac{E_L}{2}\phi^2 + 4 E_C n^2$$

The above Hamiltonian is changed when the resonator is in contact with an external environment, consisting of coupling to other quantum elements and other undesired interactions with a dissipative bath. This more general Hamiltonian can be simplified in the limit where the resonator and the quantum element are far detuned and there is weak anharmonicity. In this regime, the net effect of this interaction will be a shift in the frequency of both quantum elements due to the Lamb shift. For small couplings, the change in frequency for the resonator will be

$$\omega^{1}_R \approx \omega_R + \frac{g^2}{\Delta}$$

where $g$ is the coupling between the two elements and $\Delta$ is their difference in frequency. This frequency shift effectively allows us to measure the state of a quantum element by checking the response of its coupled resonator. This method is called *dispersive readout*.

![](../images/resonators_and_qubits.svg "Sketch of the layout of a simple chip with each qubits coupled to a resonator.")

Given this context, we see now that the scope of this experiment workflow is to characterize the resonator at a particular qubit state, typically when this is in its ground state, by looking at the response in the readout line to different frequencies. In practice, the exact response depends on the details of the circuit. Here is an example of how data would look like when several resonators are connected to the same line:

![](<../images/resonator_result.png> "Plot of Resonator spectroscopy where the different frequencies scanned are depicted on the x-axis and the absolute value of the integration is presented in the y-axis.")

### Imports

You'll start by importing the resonator spectroscopy experiment workflows from `laboneq_applications`, as well as `laboneq.simple` and a demo QPU and device setup to run in emulation mode.

In [1]:
import numpy as np
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation
from laboneq.simple import *

from laboneq_applications.experiments import (
    resonator_spectroscopy,
    resonator_spectroscopy_amplitude,
)
from laboneq_applications.qpu_types.tunable_transmon.demo_qpus import demo_platform

### QPU and Device Setup

You'll generate six qubits with pre-defined parameters, as well as a `Device_Setup` consisting of a SHFQC+, HDAWG, and PQSC. If you already have your own `DeviceSetup` and qubits configured, you'll instead initialize the session using your setup.

In [2]:
my_platform = demo_platform(6)

Then, you'll connect to the `Session`. Here we connect to an emulated one:

In [ ]:
session = Session(my_platform.setup)
session.connect(do_emulation=True)

### Running the Resonator Spectroscopy Workflow

You'll now make the experiment workflow and run:

In [ ]:
# our qubits live here in the demo setup:
qubit = my_platform.qpu.qubits[0]

my_workflow = resonator_spectroscopy.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubit=qubit,
    frequencies=np.linspace(6.8e9, 7.2e9, 1001),
)

my_results = my_workflow.run()

#### Output Simulation

You can also inspect the compiled experiment and plot the simulated output. Remember that since no kernel are used, the OutputSimulator won't show anything in the acquire line.

In [ ]:
compiled_resonator_spectroscopy = my_results.tasks["compile_experiment"].output
plot_simulation(
    compiled_resonator_spectroscopy, length=50e-6, signal_names_to_show=["measure"]
)

#### Inspecting the Source Code

You can inspect the source code of the `create_experiment` task defined in `resonator_spectroscopy` to see how the experiment pulse sequence is created:

In [ ]:
resonator_spectroscopy.create_experiment.src

### Changing the Options

We can give our Resonator spectroscopy experiment options. First, inspect what they currently are:

In [ ]:
my_new_opts = resonator_spectroscopy.experiment_workflow.options()
my_new_opts

Then provide new options. This time, you'll reduce the waiting time between shots.

In [ ]:
my_new_opts.count(2048)
my_new_opts.spectroscopy_reset_delay(100e-9)
my_new_opts

#### Run the workflow with updated options

Now, run the workflow with new options and inspect the simulated output. You'll notice that the delay between shots is now smaller.

In [ ]:
my_new_workflow = resonator_spectroscopy.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubit=qubit,
    frequencies=np.linspace(6.5e9, 7.5e9, 1001),
    options=my_new_opts,
)

my_new_results = my_new_workflow.run()
new_compiled_res_spec = my_new_results.tasks["compile_experiment"].output
plot_simulation(new_compiled_res_spec, length=50e-6, signal_names_to_show=["measure"])

### Temporary settings

The qubit parameters are used to control the settings of pulses and instruments during the experiment. We can run the resonator spectroscopy experiment with different settings by passing it a copy of the qubit with modified parameters:

In [ ]:
# Make a copy of the qubits
temp_qubits = my_platform.qpu.copy_qubits()
temp_qubit = temp_qubits[0]
# Change the value of the LO to allow for another range to be swept
temp_qubit.parameters.readout_lo_frequency = 4e9
temp_qubit.parameters.readout_resonator_frequency = 4.1e9
# increase length of the readout pulse
temp_qubit.parameters.readout_length = 10e-6

my_new_workflow = resonator_spectroscopy.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubit=temp_qubit,  # pass temporary qubits
    frequencies=np.linspace(3.5e9, 4.5e9, 1001),
    options=my_new_opts,
)

my_new_results = my_new_workflow.run()
new_compiled_res_spec = my_new_results.tasks["compile_experiment"].output
plot_simulation(new_compiled_res_spec, length=50e-6)

### Sweep the amplitude at the same time

Getting back to our found parameters, we now repeat the workflow while sweeping the amplitude at the same time. Since the analysis and the interface are different, a dedicated workflow is used, but the input only differs slightly. Consider that since the amplitude is swept in near-time, the change in amplitude will not be visible in the OutputSimulator

In [ ]:
my_workflow = resonator_spectroscopy_amplitude.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubit=qubit,
    frequencies=np.linspace(6.5e9, 7.5e9, 1001),
    amplitudes=np.linspace(0.1, 0.9, 10),
)

my_results = my_workflow.run()

Great! You've now run your Resonator Spectroscopy experiment. Check out these other experiments to keep characterizing your qubits:

In [12]:
# TODO: Add experiment links
# TODO: Add Analysis